In [2]:
# 共通で利用するライブラリ
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

# warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True, precision=3)
pd.options.display.float_format = "{:.3f}".format
pd.set_option("display.max_columns", None)
plt.style.use("seaborn")
plt.rcParams["font.size"] = 14
plt.rcParams["font.family"] = "IPAexGothic"
%matplotlib inline
%load_ext autoreload

# 追加するライブラリ
from natsort import natsorted
import glob
import os
import datetime

%autoreload
from source.data_initialization import Initialize as ini
from source.calculation import Calc
from source.reporting import Reporting
from source.make_folder import MakeFolder


## Ch.05 分析システムを構築する

データ一覧

- m_area.csv
  - 地域マスタ
  - 都道府県情報等
- m_store.csv
  - 店舗マスタ
  - 店舗名等
- tbl_order_202004.csv
  - 注文データ
  - 4月分
- tbl_order_202005.csv
  - 注文データ
  - 5月分
- tbl_order_202006.csv
  - 注文データ
  - 6月分
- tbl_order_202007.csv
  - 注文データ
  - 7月分

### Knock41: 基本的なフォルダを生成する

フォルダ構成

```bash
ch05
├── ch05.ipynb
├── source
└── data
    ├── 0_input
    ├── 10_output
    ├── 99_master
    └── race
```

In [3]:
# フォルダパスの定義

source_dir = "source"
data_dir = "data"
input_dir = os.path.join(data_dir, "0_input")
output_dir = os.path.join(data_dir, "10_output")
master_dir = os.path.join(data_dir, "99_master")
print(input_dir)


data/0_input


In [4]:
# フォルダの生成

os.makedirs(source_dir, exist_ok=True)
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)

### Knock42: 入力データのチェック機能を作る

In [5]:
# マスタデータの読み込み

m_area_file = "m_area.csv"
m_store_file = "m_store.csv"
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
print(m_area.shape)
display(m_area)

(7, 3)


,area_cd,wide_area,narrow_area
0,TK,東京,東京
1,KN,神奈川,神奈川
2,CH,千葉,千葉
3,SA,埼玉,埼玉
4,IB,北関東,茨城
5,TO,北関東,栃木
6,GU,北関東,群馬


In [6]:
# 指定した注文データ読み込み

target_year_month = "202007"
# TARGET_YEAR_MONTH = "202007"
target_file = "tbl_order_" + target_year_month + ".csv"
target_data = pd.read_csv(os.path.join(input_dir, target_file))

In [7]:
# データチェック機構（正常動作時）

max_date = pd.to_datetime(target_data["order_accept_date"]).max()
min_date = pd.to_datetime(target_data["order_accept_date"]).min()
max_str_date = max_date.strftime("%Y%m")
min_str_date = min_date.strftime("%Y%m")

if (target_year_month == min_str_date) & (target_year_month == max_str_date):
    print("OK! 日付が一致しました。")
else:
    raise print("ERROR!! 日付が一致しません。")

OK! 日付が一致しました。


In [8]:
# 当月分の初期化

target_data = ini.init_tran_df(target_data, m_store, m_area)


### Knock43: レポーティング（本部向け）を関数化する

### Knock44: レポーティング（店舗向け）を関数化する

### Knock45: 関数を実行して動作を確認する

In [9]:
# 本部向けレポート

# Reporting.make_report_headquarters(target_data, max_str_date, m_store, output_dir)

In [10]:
# 店舗向けレポート

# for store_id in m_store.loc[m_store["store_id"] != 999]["store_id"]:
#     Reporting.make_report_store(
#         target_data, store_id, m_store, max_str_date, output_dir
#     )


### Knock46: 更新に対応できる出力フォルダを作成する

In [11]:
# 新たな出力フォルダの作成

# target_output_dir = MakeFolder.make_active_folder(output_dir, target_year_month)
# data/10_output/202007_20220902_082730
target_output_dir = "data/10_output/202007_20220902_082730"


data/10_output/202007_20220902_082730


### Knock47: 都道府県別に出力できるように出力フォルダを拡張してデータを出力する

In [12]:
# 新しいフォルダに本部向けレポートを保存する

Reporting.make_report_headquarters(
    target_data, max_str_date, m_store, target_output_dir)


In [28]:
# 都道府県ごとにフォルダを動的に生成する
# 各店舗向けレポート
# data/10_output/YYYYMM_YYYYMMDD_HHMMDDSS（更新日時）/narrow_area/

for store_id in m_store.loc[m_store["store_id"] != 999]["store_id"]:
    # narrow_areaのフォルダを作成する
    area_cd = m_store.loc[m_store["store_id"] == store_id]["area_cd"]
    area_name = m_area.loc[m_area["area_cd"] == area_cd.values[0]]["narrow_area"].values[0]
    target_store_output_dir = os.path.join(target_output_dir, area_name)
    os.makedirs(target_store_output_dir,exist_ok=True)
    Reporting.make_report_store(
        target_data, store_id, m_store, max_str_date, target_store_output_dir
    )
# 5m

### Knock48: 前月のデータを動的に読み込む